In [37]:
import pandas as pd
from sklearn import cluster, mixture
import scipy
import numpy as np
import time
import collections

In [38]:
class time_est():
    def __init__(self, total_len):
        self.t_start = time.time()
        self.total_len = total_len
        self.count = 0
        self.t_ref = time.time()
    
    def check(self,no_of_check=1,info=""):
        self.count += no_of_check
        if time.time() - self.t_ref > 1:
            t_used = time.time() - self.t_start
            t_total = t_used * self.total_len / self.count
            t_remain = t_total - t_used
            process_bar = "|"
            for i in range(40):
                if (i/40) < (self.count/self.total_len):
                    process_bar += "█"
                else:
                    process_bar += " "
            process_bar += "|"
            if info != "":
                info = str(info) + "  "
            print("\r" + (str(info) + "{:.2f}% ({}/{})  ".format(self.count * 100/self.total_len, self.count,self.total_len)) 
                  + str(process_bar).ljust(45) 
                  + "Used: {:02.0f}:{:02.0f}:{:02.0f}".format(int(t_used/3600), int(t_used/60)%60, t_used % 60).ljust(16) 
                  + "ETA: {:02.0f}:{:02.0f}:{:02.0f}".format(int(t_remain/3600), int(t_remain/60)%60, t_remain % 60),end="")
            self.t_ref = time.time()
        if self.count == self.total_len:
            t_used = time.time() - self.t_start
            if info != "":
                info = str(info) + "  "
            print("\r" + str(info) + "Finished in " 
                  + "{:02.0f}:{:02.0f}:{:02.0f}".format(int(t_used/3600), int(t_used/60)%60, t_used % 60).ljust(100))
    def get(self,no_of_check=1):
        process_bar = "|"
        for i in range(40):
            if (i/40) < (self.count/self.total_len):
                process_bar += "█"
            else:
                process_bar += " "
        process_bar += "|"
        self.count += no_of_check
        t_used = time.time() - self.t_start
        t_total = t_used * self.total_len / self.count
        t_remain = t_total - t_used
        return "{} ETA: {:02.0f}:{:02.0f}:{:02.0f}".format(process_bar, int(t_remain/3600), int(t_remain/60)%60, t_remain % 60)
        

In [39]:
df= pd.read_csv('top100_rating.csv')

In [40]:
df_np = np.array(df)

In [41]:
artist_list = list(set(df["artist"]))
customer_id_list = list(set(df["customer_id"]))

In [42]:
customer_id_dict = collections.defaultdict()
est = time_est(len(customer_id_list))
for customer_id in customer_id_list:
    customer_id_dict[customer_id] = collections.defaultdict()
    est.check()
est = time_est(len(df_np))   
for customer_id, rating, artist in df_np:
    try:
        customer_id_dict[customer_id][artist] = int(rating)
    except:
        pass
    est.check()

Finished in 00:00:04                                                                                            
Finished in 00:00:10                                                                                            


In [43]:
est = time_est(len(customer_id_dict.keys()))
for customer_id in customer_id_dict:
    sum_rating = 0
    for artist in customer_id_dict[customer_id]:
#         customer_id_dict[customer_id][artist] = np.exp(customer_id_dict[customer_id][artist]/100)
        sum_rating += customer_id_dict[customer_id][artist]
    for artist in customer_id_dict[customer_id]:
        customer_id_dict[customer_id][artist] /= sum_rating
    est.check()

Finished in 00:00:03                                                                                            


In [44]:
np.save("customer_id_dict.npy", customer_id_dict)

In [45]:
customer_id_dict = np.load("customer_id_dict.npy").item()

In [46]:
artist_dict = collections.defaultdict()

est = time_est(len(customer_id_dict.keys()))
for customer_id in customer_id_dict:
    for artist in customer_id_dict[customer_id]:
        try:
            artist_dict[artist][customer_id] = customer_id_dict[customer_id][artist]
        except:
            artist_dict[artist] = collections.defaultdict()
            artist_dict[artist][customer_id] = customer_id_dict[customer_id][artist]
    est.check()

Finished in 00:00:03                                                                                            


In [47]:
np.save("artist_dict.npy", artist_dict)